In [1]:
from collections import deque
from itertools import accumulate


class Automato:
    """Implementa um automato finito determinístico (AFD) que obedece a 
    seguinte definição:
    M = (Q, Σ, δ, q0, F)
    Q: Conjunto finito de estados
    Σ: Alfabeto
    δ: Função de transição
    q0: Estado inicial
    F: Conjunto de estados finai
    """
    
    estados_finitos: set[str]
    alfabeto: set[str]
    transicoes: dict[dict[str, str]]
    estado_inicial: str
    conj_estados_finais: set[str]
    fita: deque[str]

    def __init__(self, estados_finitos, alfabeto, transicoes,
                estado_inicial, conj_estados_finais):
        self.estados_finitos: set = estados_finitos
        self.alfabeto: set = alfabeto
        self.transicoes = transicoes
        self.estado_inicial = estado_inicial
        self.estados_finais = conj_estados_finais
        self.fita = deque()
        self.cursor = 0
        self.maquina_estados: MaquinaEstados = MaquinaEstados(self)

    def __repr__(self) -> str:
        return f"""\
        M = (Q, Σ, δ, q0, F)
        Q: {{{','.join(self.estados_finitos)}}}
        Σ: {{{self.alfabeto}}}
        δ: {self.maquina_estados.transicao}
        q0: {self.estado_inicial}
        F: {self.estados_finais}\
        """
    
    def __eq__(self, o: "Automato") -> bool:
        """Retorna True se os dois automatos forem equivalentes."""
        # Aqui é preciso aplicar a técnica de minimização em ambos 
        # automatos, o que implica implementar 1. os algoritmos de remoção
        # de transições em vazio; 2. de remoção de não-determinismos;
        # 3. de remoção de estados inúteis ou inacessíveis, e o
        # algoritmo de minimização, nesta ordem.
        #
        # minimo_self = self.rm_transicoes_vazio().rm_nao_determinismo().rm_estados_inuteis()
        # minimo_o = o.rm_transicoes_vazio().rm_nao_determinismo().rm_estados_inuteis()
        pass

    def escrever_fita(self, cadeia: str) -> None:
        """Escreve uma cadeia de caracteres na fita."""
        self.fita.clear()
        self.fita = deque(cadeia)
        self.fita.append("\0")
    
    def ler_celula(self) -> str:
        """Retorna o conteudo da celula atual da fita."""
        self.cursor += 1
        return self.fita.pop()

    def reconhecer_cadeia(self, cadeia) -> bool:
        """Recebe uma cadeia de caracteres e retorna True se a cadeia 
        for aceita pelo automato.
        """
        return self.maquina_estados.reconhecer_cadeia(cadeia)
    
    def rm_transicoes_vazio(self) -> "Automato":
        """Retorna um novo automato sem transições em vazio."""
        # TODO
        pass

    def rm_nao_determinismo(self) -> "Automato":
        """Retorna um novo automato sem não-determinismos."""
        # TODO
        pass

    def rm_estados_inuteis(self) -> "Automato":
        """Retorna um novo automato sem estados inuteis."""
        # TODO
        pass


#Maquina de estados
class MaquinaEstados():
    """Classe que implementa uma máquina de estados finitos 
    determinística por composição da classe Automato.
    """
    
    funcao_transicao: dict[dict[str, str]]
    estado_inicial: str
    estados_finais: set[str]


    def __init__(self, automato: Automato):
        self.transicao = automato.transicoes
        self.estados_finais = automato.estados_finais
        self.estado_inical = automato.estado_inicial
        self.__estado_atual = automato.estado_inicial
        self.cadeia_restante = None
        self.posicao_cursor = 0
        self.movimentos = list()
    
    def config_seguinte(self, simbolo):
        """Aplica a função de transição δ(p, σ) -> q."""
        self.__estado_atual = self.transicao[self.__estado_atual][simbolo]

    # Opcao com recurcao
    def reconhecer_cadeia(self, cadeia):
        """Reconhece uma cadeia inteira na fita, de acordo com a 
        função de transição.
        """

        if self.cadeia_restante == None:
            self.movimentos = []
            self.movimentos.append((self.__estado_atual, cadeia))
        
        resultado = False
        if (self.__estado_atual, cadeia[0]) in self.transicao.keys():
        
            self.__estado_atual = self.transicao[(self.__estado_atual, cadeia[0])]
            self.cadeia_restante = cadeia[1:]
            self.movimentos.append((self.__estado_atual, self.cadeia_restante))
            if self.cadeia_restante == "":
                return True if self.__estado_atual in self.estados_finais else False
            else:
                resultado = self.reconhecer_cadeia(self.cadeia_restante)
        else:            
            return False
        self.__estado_atual = self.estado_inical
        self.cadeia_restante = None
        return resultado


In [2]:
estados = set(["q0", "q1", "q2", "q3"])
alfabeto = set(['0', '1', '2'])
funcao_de_transicao = {
    ('q0', '0'): 'q0',
    ('q0', '1'): 'q1',
    ('q0', '2'): 'q3',
    ('q1', '0'): 'q3',
    ('q1', '1'): 'q1',
    ('q1', '2'): 'q2',
    ('q2', '0'): 'q3',
    ('q2', '1'): 'q3',
    ('q2', '2'): 'q2',
    ('q3', '0'): 'q3',
    ('q3', '1'): 'q3',
    ('q3', '2'): 'q3',
}
estado_inicial = 'q0'
estados_finais = set(['q1', 'q2'])

In [3]:
automato_1 = Automato(estados, alfabeto, funcao_de_transicao, estado_inicial, estados_finais)

In [4]:
print(automato_1.reconhecer_cadeia('0001112'))

True


In [5]:
automato_1.escrever_fita('0001112')
print(automato_1.fita)

deque(['0', '0', '0', '1', '1', '1', '2', '\x00'])


In [6]:
from collections import deque

def escrever_fita(fita: deque, cadeia: str) -> None:
    """Escreve uma cadeia de caracteres na fita."""
    fita.clear()
    fita = deque(cadeia)
    fita.append("\0")
    return fita

fita = deque()
fita = escrever_fita(fita, "0001112")
fita.append("\0")
print(fita)


deque(['0', '0', '0', '1', '1', '1', '2', '\x00', '\x00'])


In [7]:
funcao_de_transicao2 = {
    'q0': {
        '0': 'q0',
        '1': 'q1',
        '2': 'q3',
    },
    'q1': {
        '0': 'q3',
        '1': 'q1',
        '2': 'q2',
    },
    'q2': {
        '0': 'q3',
        '1': 'q3',
        '2': 'q2',
    },
    'q3': {
        '0': 'q3',
        '1': 'q3',
        '2': 'q3',
    },    
}

In [8]:
from itertools import accumulate
from collections import deque
from functools import reduce

def muda_estado(estado_atual, simbolo) -> str:
    estado_atual = funcao_de_transicao2[estado_atual][simbolo]
    return estado_atual

estado_atual = "q0"
estado_atual = muda_estado(estado_atual, "2")
print(estado_atual)


q3


In [9]:
cadeia = deque("0001112")
transicoes = accumulate(cadeia, muda_estado, initial='q0')
transicoes2 = reduce(muda_estado, cadeia, 'q0')
# print(list(transicoes))


In [10]:
print(automato_1.reconhecer_cadeia("0011222"))
print(automato_1.maquina_estados.movimentos)
print(str(automato_1.maquina_estados.movimentos).replace("), ", ") ⊢ "))



True
[('q0', '0011222'), ('q0', '011222'), ('q0', '11222'), ('q1', '1222'), ('q1', '222'), ('q2', '22'), ('q2', '2'), ('q2', '')]
[('q0', '0011222') ⊢ ('q0', '011222') ⊢ ('q0', '11222') ⊢ ('q1', '1222') ⊢ ('q1', '222') ⊢ ('q2', '22') ⊢ ('q2', '2') ⊢ ('q2', '')]


In [15]:
import pandas as pd
import numpy as np

df = np.array(funcao_de_transicao2)
df.T
# testess

array({'q0': {'0': 'q0', '1': 'q1', '2': 'q3'}, 'q1': {'0': 'q3', '1': 'q1', '2': 'q2'}, 'q2': {'0': 'q3', '1': 'q3', '2': 'q2'}, 'q3': {'0': 'q3', '1': 'q3', '2': 'q3'}},
      dtype=object)